In [2]:
import pandas as pd

# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# If needed, also expand the width
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)  # or a large int

from copy import deepcopy
from random import randint, choice
import random
import numpy as np

# from library.solution import Solution
# from library.algorithms.hill_climbing import hill_climbing
# from library.algorithms.simulated_annealing import simulated_annealing
from library.wedding_solution_list import Wedding_Solution

In [ ]:
from library.solution import Solution
import pandas as pd
# from random import randint
# import random
fitness_grid = pd.read_csv("library/wedding_seat_data.csv")

class Wedding_Solution(Solution):
    def __init__(self, repr=None, tables=8, attendees=64, values_grid=fitness_grid):
        self.tables = tables
        self.attendees = attendees
        self.values_grid = values_grid
        self.seats_per_table = int(attendees / tables)

        if repr:
            repr = self._validate_repr(repr)

        super().__init__(repr=repr)

    # only used to validate the given representation (random initialization already follows these rules)
    def _validate_repr(self, repr):
        # repr needs to be a dictionary
        if not isinstance(repr, list):
            raise TypeError("Representation must be a list.")
        
        # this ensures that the partitions are always even and the same amounts
        if not (self.attendees / self.tables).is_integer():
            raise ValueError("The number of attendees and tables for this solution must divide evenly into each other.")
        
        # all the values in the repr need to be an int, if not - change to an int
        if not all([isinstance(table_num, int) for table_num in repr]):
            repr = [int(table) for table in repr]

        # the number of unique numbers in repr need to be the same as the number of tables defined
        if len(set(repr)) != self.tables:
            raise ValueError("Missing a table number from representation assignment: Table(s)", set(range(1,self.tables+1)) - set(repr))
        
        # there needs to be the same number of ppl at each table
        if not all(repr.count(x) == self.seats_per_table for x in range(1,self.tables)):
            raise ValueError("The number of people assigned to each table need to be the same.")
        
        # the length of all the attendees need to match the defined variable for attendees
        if len(repr) != self.attendees:
            raise ValueError("The total number of attendees in the representation need to match the defined number of attendees")
        
        return repr

    # Override the superclass's methods
    def random_initial_representation(self):
        seats = []
        for i in range(1,self.tables+1): # generates the same number of numbers in a row ([1] * 5 = [1,1,1,1,1])
            seats = seats + [i] * self.seats_per_table
        random.shuffle(seats) # randomly assign tables to participants
        return seats

    # returns the fitness of the representation / solution
    def fitness(self):
        fitness = 0
        for table_num in range(1, self.tables+1): # loop through the tables
            fitness += self.table_fitness(table_num)

        return fitness
    
    # checks the fitness of a specific table instead of all the tables
    def table_fitness(self, table):
        fitness = 0
        table_seats = [i for i, x in enumerate(self.repr) if x == table]
        table_seats = [x + 1 for x in table_seats] # add 1 to everything to match the fitness grid
        people_seen = []
        for personA in table_seats: # loop through the people at the table 
            people_seen.append(personA) # make sure to skip personA's score with themself
            for personB in table_seats: # compare person A with everyone else (personB) at the table
                if personB not in people_seen: # skip the people that were already counted to not count them twice
                    fitness += self.values_grid[self.values_grid['idx'] == personA][str(personB)].values[0]
        return fitness
    
    # clearly shows which person sits at which table for debugging purposes
    def pretty_print(self):
        tables = {i: [] for i in range(1, self.tables+1)}  # Tables 1 through 8

        for idx, table in enumerate(self.repr):
            tables[table].append(idx)

        print("----------SEATING ARRANGEMENTS----------")
        for table_num in range(1, self.tables+1): 
            # checks to see if its single digit - if so, add a space for cleaner reading
            # also adds 1 so that it is clear which person is being referred to
            indices = [f"{i+1:2}" for i in tables[table_num]]
            print(f"Table {table_num}: [{', '.join(indices)}]")
        print("----------------------------------------")

In [3]:
tables = 4
attendees = 12
initial_solution = [1, 3, 1, 1, 4, 2, 4, 4, 3, 3, 2, 2]
repr_initial_solution = Wedding_Solution(repr=initial_solution, tables=tables, attendees=attendees)
print("ASSIGNED INITIALIZATION: ")
print("Repr: ", repr_initial_solution)
print("Seats per table: ", repr_initial_solution.seats_per_table)
print("Fitness: ", repr_initial_solution.fitness())

print("\nRANDOM INITIALIZATION: ")
repr = Wedding_Solution(tables=tables, attendees=attendees)
print("Repr: ", repr)
print("Seats per table: ", repr.seats_per_table)
print("Fitness: ", repr.fitness())

ASSIGNED INITIALIZATION: 
Repr:  [1, 3, 1, 1, 4, 2, 4, 4, 3, 3, 2, 2]
Seats per table:  3
Fitness:  7000

RANDOM INITIALIZATION: 
Repr:  [3, 2, 1, 3, 2, 4, 1, 1, 4, 3, 2, 4]
Seats per table:  3
Fitness:  2600


In [4]:
class WeddingGASolution(Wedding_Solution):
    def __init__(
        self,
        mutation_function,
        crossover_function,
        repr = None
    ):
        super().__init__(
            repr=repr,
        )

        # Save as attributes for access in methods
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function

    def mutation(self, mut_prob):
        # Apply mutation function to representation
        new_repr = self.mutation_function(self.repr, mut_prob)
        # Create and return individual with mutated representation
        return WeddingGASolution(
            selection_function = self.selection_function,
            mutation_function=self.mutation_function,
            crossover_function=self.crossover_function,
            repr=new_repr
        )

    def crossover(self, other_solution):
        # Apply crossover function to self representation and other solution representation
        offspring1_repr, offspring2_repr = self.crossover_function(self.repr, other_solution.repr)

        # Create and return offspring with new representations
        return (
            WeddingGASolution(
                selection_function = self.selection_function,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring1_repr
            ),
            WeddingGASolution(
                selection_function = self.selection_function,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring2_repr
            )
        )

In [5]:
repr = Wedding_Solution()
print(repr)
repr.pretty_print()

[8, 6, 7, 5, 5, 3, 4, 4, 4, 3, 1, 7, 1, 3, 1, 4, 3, 3, 4, 7, 5, 8, 8, 1, 6, 8, 8, 8, 3, 5, 7, 6, 7, 5, 7, 3, 4, 6, 2, 5, 4, 1, 2, 6, 8, 3, 6, 8, 7, 2, 5, 6, 2, 2, 2, 4, 7, 1, 1, 5, 1, 6, 2, 2]
----------SEATING ARRANGEMENTS----------
Table 1: [11, 13, 15, 24, 42, 58, 59, 61]
Table 2: [39, 43, 50, 53, 54, 55, 63, 64]
Table 3: [ 6, 10, 14, 17, 18, 29, 36, 46]
Table 4: [ 7,  8,  9, 16, 19, 37, 41, 56]
Table 5: [ 4,  5, 21, 30, 34, 40, 51, 60]
Table 6: [ 2, 25, 32, 38, 44, 47, 52, 62]
Table 7: [ 3, 12, 20, 31, 33, 35, 49, 57]
Table 8: [ 1, 22, 23, 26, 27, 28, 45, 48]
----------------------------------------


In [14]:
mut_prob = 0.5
x = 0.62#random.random()

# if x <= mut_prob:
#     print("Entered.")
# else:
#     print("Didn't Enter.")

if x > mut_prob:
    print("Didn't Enter.")
else:
    print("Entered.")

print("Chosen x: ", round(x,2))

Didn't Enter.
Chosen x:  0.62


In [ ]:
# make it genuinly greedy by starting to make personB the max of the ppl with relationships (sort descending)
def greedy_swap_mutation(representation, attendees_num, mut_prob, fitness_grid):
    new_repr = deepcopy(representation)

    # random chance to do the mutation
    if random.random() >= mut_prob:
        print("Randomly chosen to not implement.")
        return new_repr

    # selects random different tables to swap from, 
    randomPersonA = randint(1, attendees_num)
    randomTableA = new_repr[randomPersonA-1]

    # get positivley valued neighbors
    filtered = fitness_grid[fitness_grid['idx'] == randomPersonA].iloc[0]
    filtered = filtered.drop('idx')
    peopleWithRelationship = [int(x) for x in filtered[filtered > 0].index] # only getting the attendees that are valued by randomPersonA

    # determine person B (don't choose someone from the same table)
    randomPersonB = None
    for personB in peopleWithRelationship:
        randomPersonB = personB
        randomTableB = new_repr[personB-1]
        if randomTableA == randomTableB:
            continue
    
    # if there are no values people that don't share the same table as A, randomly choose B
    if randomPersonB == None:
        randomPersonB = randint(1, attendees_num)
        randomTableB = new_repr[randomPersonB-1]
        while randomTableB == randomTableA: # always choose a different table
            randomPersonB = randint(1, attendees_num)
            randomTableB = new_repr[randomPersonB-1]

    # Swap the two people between the tables
    new_repr[randomPersonA-1] = randomTableB
    new_repr[randomPersonB-1] = randomTableA

    print("Swapped person #" + str(randomPersonA) + " at table #" + str(randomTableA) + " with person #" + 
          str(randomPersonB) + " at table #" + str(randomTableB))

    return Wedding_Solution(repr=new_repr)

In [ ]:
swap_repr = greedy_swap_mutation(repr.repr, repr.tables, repr.attendees, 1, repr.values_grid)
print(swap_repr.repr)
print(repr.repr)

Swapped person #59 at table #4 with person #9 at table #7


In [ ]:
def scramble_mutation(representation, attendees_num, mut_prob):
    new_repr = deepcopy(representation)

    # random chance to do the mutation
    if random.random() >= mut_prob:
        print("Randomly chosen to not implement.")
        return new_repr

    # choose a random number of people to get a subset
    numOfPeopleToShuffle = randint(2, attendees_num) # cant shuffle 1 person, nothing would change

    startPerson = randint(0, attendees_num-numOfPeopleToShuffle) # set the biggest number it can randomly choose as the total - the number aimed to shuffle
    stopPerson = startPerson + numOfPeopleToShuffle
    
    print("Number to shuffle: ", numOfPeopleToShuffle)
    print("Range to shuffle: " + str(startPerson) + " - " + str(stopPerson))

    # shuffle the subset
    peopleToShuffle = new_repr[startPerson:stopPerson]
    random.shuffle(peopleToShuffle)
    new_repr[startPerson:stopPerson] = peopleToShuffle

    return Wedding_Solution(repr=new_repr)

In [ ]:
scramble_repr = scramble_mutation(repr.repr, repr.attendees, 1)
print(scramble_repr.repr)
print(repr.repr)

Number to shuffle:  35
Range to shuffle: 5 - 40


In [ ]:
def inversion_mutation(representation, attendees_num, mut_prob):
    new_repr = deepcopy(representation)
    
    # random chance to do the mutation
    if random.random() >= mut_prob:
        print("Randomly chosen to not implement.")
        return new_repr
    
    # choose a random number of people to get a subset
    numOfPeopleToInvert = randint(2, attendees_num) # cant invert 1 person, nothing would change

    startPerson = randint(0, attendees_num-numOfPeopleToInvert) # set the biggest number it can randomly choose as the total - the number aimed to shuffle
    stopPerson = startPerson + numOfPeopleToInvert

    print("Number to invert: ", numOfPeopleToInvert)
    print("Range to invert: " + str(startPerson) + " - " + str(stopPerson))

    # shuffle the subset
    peopleToInvert = new_repr[startPerson:stopPerson]
    print(peopleToInvert)
    peopleToInvert.reverse()
    print(peopleToInvert)
    print("---------")
    new_repr[startPerson:stopPerson] = peopleToInvert
    
    return Wedding_Solution(repr=new_repr)

In [485]:
inversion_repr = inversion_mutation(repr.repr, repr.attendees, 1)
print(inversion_repr.repr)
print(repr.repr)

Number to invert:  9
Range to invert: 13 - 22
[6, 6, 2, 4, 7, 5, 3, 3, 6]
[6, 3, 3, 5, 7, 4, 2, 6, 6]
---------
[8, 7, 5, 4, 5, 8, 6, 1, 7, 6, 8, 4, 3, 6, 3, 3, 5, 7, 4, 2, 6, 6, 2, 5, 1, 1, 2, 7, 7, 6, 7, 3, 3, 3, 4, 7, 5, 8, 1, 8, 5, 3, 2, 5, 6, 2, 4, 5, 6, 1, 8, 7, 1, 2, 3, 1, 8, 4, 4, 1, 8, 2, 2, 4]
[8, 7, 5, 4, 5, 8, 6, 1, 7, 6, 8, 4, 3, 6, 6, 2, 4, 7, 5, 3, 3, 6, 2, 5, 1, 1, 2, 7, 7, 6, 7, 3, 3, 3, 4, 7, 5, 8, 1, 8, 5, 3, 2, 5, 6, 2, 4, 5, 6, 1, 8, 7, 1, 2, 3, 1, 8, 4, 4, 1, 8, 2, 2, 4]


### Crossover

In [ ]:
repr_2 = Wedding_Solution()
print(repr_2)
repr_2.pretty_print()

[4, 3, 8, 4, 3, 7, 4, 7, 6, 8, 3, 1, 2, 3, 6, 1, 2, 5, 7, 4, 3, 2, 2, 8, 6, 2, 5, 1, 7, 7, 5, 4, 7, 1, 5, 5, 1, 3, 6, 3, 4, 6, 1, 8, 8, 1, 5, 5, 2, 6, 6, 5, 8, 2, 8, 6, 4, 8, 7, 1, 3, 4, 7, 2]
----------SEATING ARRANGEMENTS----------
Table 1: [12, 16, 28, 34, 37, 43, 46, 60]
Table 2: [13, 17, 22, 23, 26, 49, 54, 64]
Table 3: [ 2,  5, 11, 14, 21, 38, 40, 61]
Table 4: [ 1,  4,  7, 20, 32, 41, 57, 62]
Table 5: [18, 27, 31, 35, 36, 47, 48, 52]
Table 6: [ 9, 15, 25, 39, 42, 50, 51, 56]
Table 7: [ 6,  8, 19, 29, 30, 33, 59, 63]
Table 8: [ 3, 10, 24, 44, 45, 53, 55, 58]
----------------------------------------


### Below not working still

In [ ]:
# def standard_crossover(parent1_repr, parent2_repr, attendees, mut_prob):

#     crossoverPoint = randint(1, attendees-1) # cant be 0
#     print("CrossOver Point: " + str(crossoverPoint))
#     child1_repr = parent1_repr[:crossoverPoint] + parent2_repr[crossoverPoint:]
#     child2_repr = parent2_repr[:crossoverPoint] + parent1_repr[crossoverPoint:]

#     print("Child1: ",child1_repr)
#     print("Child2: ",child2_repr)

#     return [Wedding_Solution(repr=child1_repr), Wedding_Solution(repr=child2_repr)]

In [ ]:
# inversion_repr = standard_crossover(repr.repr, repr_2.repr, repr.attendees, 1)
# # print(inversion_repr)
# print(repr.repr)
# print(repr_2.repr)
# print("-------")
# print(inversion_repr[0].repr)
# print(inversion_repr[1].repr)

In [ ]:
# def custom_crossover(parent1_repr, parent2_repr):
#     return